In [1]:
import dask
import dask.dataframe as dd
import pandas as pd
from bs4 import BeautifulSoup
import re
import requests
from zipfile import ZipFile
import os
import numpy as np

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=8, processes=False, memory_limit='48GB')
client

Client Scheduler: inproc://192.168.1.8/253/1 Dashboard: http://192.168.1.8:8787/status,Cluster Workers: 1 Cores: 8 Memory: 48.00 GB


In [4]:
DATA_DIR = './data'
XML_DATA_URL = 'https://afdata.s3.us-gov-west-1.amazonaws.com'

In [5]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=chunk_size):
            fd.write(chunk)
            
def read_files(path, ext):
    file_list = []
    for root, folders, docs in os.walk(path):
        file_list.extend( [os.path.join(root, doc) for doc in docs if ext in doc] )

    return file_list

In [6]:
# Download XML list of data sources.
xml_data_path = DATA_DIR+'/data_sources.xml'
xml_data = download_url(XML_DATA_URL, xml_data_path)

In [34]:
# Download all zip data files from the XML source.
with open(xml_data_path, 'r') as xml:
    soup = BeautifulSoup(xml, 'xml')
    
    # XML structure: <Contents><Key>filename</Key><Size>bytes</Size></Contents>
    contents_elements = soup.find_all('Contents')
    
    for contents in contents_elements:
        key = contents.find('Key')
        filename = str(key.text)
        if not re.search(r'\.zip$', filename):
            continue
        
        save_path = str.format('{0}/{1}', DATA_DIR, filename)
        url = str.format('{0}/{1}', XML_DATA_URL, filename)     
        expected_size = int(str(contents.find('Size').text))
        
        # Only download if the file doesn't exist with the expected size in bytes.
        if os.path.exists(save_path):
            actual_size = os.path.getsize(save_path)
            if expected_size == actual_size:
                print(str.format('{0} at {1} bytes already exists.', filename, expected_size))
                continue
        
        print(str.format('Downloading {0}...', url))
        
        download_url(url, save_path)
        actual_size = os.path.getsize(save_path)
        
        if actual_size != expected_size:
            print(str.format('WARNING: File size for {0} at {1} bytes does not match the expected size of {2} bytes.',
                            filename, actual_size, expected_size))
        else:  
            print(str.format('Successfully downloaded {0} to {1}. Filesize: {2} bytes.'
                             , url, save_path, actual_size))
        
    xml.close()

Scenario_Data/AIS/AIS_2015_01_Zone01.zip at 699204 bytes already exists.
Scenario_Data/AIS/AIS_2015_01_Zone02.zip at 4800320 bytes already exists.
Scenario_Data/AIS/AIS_2015_01_Zone03.zip at 50471539 bytes already exists.
Scenario_Data/AIS/AIS_2016_01_Zone01.zip at 434183 bytes already exists.
Scenario_Data/AIS/AIS_2016_01_Zone02.zip at 3169488 bytes already exists.
Scenario_Data/AIS/AIS_2016_01_Zone03.zip at 55155734 bytes already exists.
Scenario_Data/AIS/AIS_2017_01_Zone01.zip at 176297 bytes already exists.
Scenario_Data/AIS/AIS_2017_01_Zone02.zip at 3566591 bytes already exists.
Scenario_Data/AIS/AIS_2017_01_Zone03.zip at 52487368 bytes already exists.
Scenario_Data/AIS/Zone10_2009_01.zip at 520366084 bytes already exists.
Scenario_Data/AIS/Zone10_2010_01.zip at 503698861 bytes already exists.
Scenario_Data/AIS/Zone10_2011_01.gdb.zip at 425399907 bytes already exists.
Scenario_Data/AIS/Zone10_2012_01.gdb.zip at 603860220 bytes already exists.
Scenario_Data/AIS/Zone10_2013_01.gdb.z

In [42]:
# Unzip all data files.
for path in read_files(path=DATA_DIR, ext='.zip'):
    extract_path = '/'.join(str(path).rsplit('/')[:-1]) + '/unzipped/'
    with ZipFile(path, 'r') as zipfile:
        zipfile.extractall(extract_path)

In [ ]:
# Clean TLE data, Save as pipe delimitted datasets.
tle_files = read_files('./data/Scenario_Data/TLE/unzipped', '.txt')
for path in tle_files:
    filename = re.sub(r'\.txt$', '.clean.txt', path)
    with open(path, 'r') as file:
        with open(filename, 'w', encoding='utf-8') as newfile:
            while True:
                line = file.readline()
                if not line:
                    file.close()
                    newfile.close()
                    print(filename)
                    break
                
                newline = ''
                line2 = False
                if re.search(r'^2', line):
                    newline = '\r\n'
                    line2 = True
   
                line = re.sub(r'^(1|2)\s+', '', line)
                line = re.sub(r'(^\s+|\s+$)', '', line)
                line = re.sub(r'\\', '|', line)
                line = re.sub(r'\s+', '|', line)   

                # Sometimes Mean Motion and Revolution number are crammed together. Separate.
                rx = r'(?<=\|)\d{1,2}\.\d{13}'
                if line2 and re.search(r'(?<=\|)\d{1,2}\.\d{13}', line):
                    last_num = re.findall(rx, line)[0]
                    mean_motion = last_num[:-5]
                    rev_num = last_num[-5:]
                    # print(mean_motion, rev_num)
                    line = re.sub(rx, mean_motion+'|'+rev_num, line)
                    
                line = line + newline
                newfile.write(line)

./data/Scenario_Data/TLE/unzipped/tle2004_1of8.clean.txt
./data/Scenario_Data/TLE/unzipped/tle2004_2of8.clean.txt
./data/Scenario_Data/TLE/unzipped/tle2004_3of8.clean.txt
./data/Scenario_Data/TLE/unzipped/tle2004_4of8.clean.txt
./data/Scenario_Data/TLE/unzipped/tle2004_5of8.clean.txt


In [11]:
tle_dtypes = {
    'SatNum': str
    , 'IntnlDesignator': str
    , 'EpochYear': np.float
    , 'JulianDayFrac':  np.float
    , 'BallisticCoef': np.float
    , 'SecDerivMeanMotion': str
    , 'DragTerm': str
    , 'EphemerisType': np.int
    , 'ElemNumCheckSum': np.int
    , 'SatNum2': str
    , 'Inclination': np.float
    , 'RightAscension': np.float
    , 'Eccentricity': str
    , 'ArgPerigree': np.float
    , 'MeanAnomaly': np.float
    , 'MeanMotion': np.float
    , 'RevNumEpochCheckSum': np.int
}

tle_columns = [col for col in tle_dtypes]

In [31]:
tle_data = dd.read_csv('./data/Scenario_Data/TLE/unzipped/*.clean.txt'
                       , sep='|', header=None, names=tle_columns, dtype=tle_dtypes, encoding='utf-8')

In [34]:
tle_data.describe()

,EpochYear,JulianDayFrac,DragTerm,EphemerisType,ElemNumCheckSum,SatNum2,Inclination,RightAscension,Eccentricity,ArgPerigree,MeanAnomaly,MeanMotion,RevNumEpochCheckSum
npartitions=1,,,,,,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [9]:
pd.read_csv?

Signature:
dd.read_csv(
    urlpath,
    blocksize='default',
    lineterminator=None,
    compression=None,
    sample=256000,
    enforce=False,
    assume_missing=False,
    storage_options=None,
    include_path_column=False,
    **kwargs,
)
Docstring:
Read CSV files into a Dask.DataFrame

This parallelizes the :func:`pandas.read_csv` function in the following ways:

- It supports loading many files at once using globstrings:

    >>> df = dd.read_csv('myfiles.*.csv')  # doctest: +SKIP

- In some cases it can break up large files:

    >>> df = dd.read_csv('largefile.csv', blocksize=25e6)  # 25MB chunks  # doctest: +SKIP

- It can read CSV files from external resources (e.g. S3, HDFS) by
  providing a URL:

    >>> df = dd.read_csv('s3://bucket/myfiles.*.csv')  # doctest: +SKIP
    >>> df = dd.read_csv('hdfs:///myfiles.*.csv')  # doctest: +SKIP
    >>> df = dd.read_csv('hdfs://namenode.example.com/myfiles.*.csv')  # doctest: +SKIP

Internally ``dd.read_csv`` uses :func:`pandas.read

In [12]:
dd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]],
    sep=',',
    delimiter=None,
    header='infer',
    names=None,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    skipfooter=0,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal: str = '.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
    escapechar=None,
    comment=None,
    encoding=None,
    dialect=None,
    error_bad_lines=True,
    warn_bad_l